In [55]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import string

In [56]:
item_detail_list_input_path = './data/intergrated_data/item/detail_list.csv'
item_id_list_input_path = './data/intergrated_data/item/id_list.csv'
item_overall_list_input_path = './data/intergrated_data/item/overall_list.csv'
item_rating_list_input_path = './data/intergrated_data/rating/rating_list.csv'

In [57]:
if not os.path.exists('./data/datasets'):
    os.makedirs('./data/datasets')


In [58]:
item_detail_list_out_path = "./data/datasets/item_{}/detail_list.csv"
item_id_list_out_path = "./data/datasets/item_{}/id_list.csv"
item_overall_list_out_path = "./data/datasets/item_{}/overall_list.csv"
item_rating_list_out_path = "./data/datasets/rating_{}/rating_list.csv"

In [59]:
details = pd.read_csv(item_detail_list_input_path, sep=',', encoding='utf-8')
ids = pd.read_csv(item_id_list_input_path, sep=',', encoding='utf-8')
overalls = pd.read_csv(item_overall_list_input_path, sep=',', encoding='utf-8')
ratings = pd.read_csv(item_rating_list_input_path, sep=',', encoding='utf-8')

In [60]:
details.drop_duplicates(keep='first', inplace=True)

In [61]:
details = details[details['price'] >= 0]

In [62]:
details = details[details['image_url'] != -1]

In [63]:
def format_cate(str):
    str = str.lower()
    str = str.strip()
    return str

In [64]:
details['category'] = details['category'].apply(format_cate)

In [65]:
genres = set()
for i in details['category']:
    genre_group = i.split("/")
    for genre in genre_group:
        genres.add(genre)

In [66]:
cates = pd.DataFrame(columns=['id', 'category'])
cates['id'] = np.arange(len(genres)).tolist()
cates['category'] = list(genres)
cates.to_csv('./data/datasets/categories.csv', sep=',', encoding='utf-8', index=False)

In [67]:
genres = list(genres)
for genre in genres:
    details[genre] = np.zeros(len(details))

for i in range(len(details)):
    genre_group = details.iloc[i]['category']
    for genre in genre_group.split("/"):
        details.at[i, genre] = 1

C:\Users\Pc\AppData\Local\Temp/ipykernel_13316/1343345615.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  details[genre] = np.zeros(len(details))


In [68]:
ids.drop_duplicates(keep='first', inplace=True)

In [69]:
overalls.dropna(inplace=True)

In [70]:
overalls.drop_duplicates(keep='first', inplace=True)

In [71]:
overalls = overalls[overalls['avg_rating'] >= 0]
overalls = overalls[overalls['n_reviews'] >= 0]

In [72]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875864 entries, 0 to 875863
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    875864 non-null  int64 
 1   item_id    875864 non-null  int64 
 2   rating     875864 non-null  int64 
 3   timestamp  875864 non-null  int64 
 4   comment    564395 non-null  object
dtypes: int64(4), object(1)
memory usage: 33.4+ MB


In [73]:
ratings.drop_duplicates(subset=['user_id', 'item_id'], keep='first', inplace=True)

In [74]:
def scale_rating(x):
    if (x <= 2):
        return 1
    if (x == 3):
        return 2
    elif (x == 4):
        return 3
    else:
        return 4

In [75]:
ratings['rating'] = ratings['rating'].apply(scale_rating)

In [76]:
print(len(ratings[ratings['rating'] == 1]))
print(len(ratings[ratings['rating'] == 2]))
print(len(ratings[ratings['rating'] == 3]))
print(len(ratings[ratings['rating'] == 4]))

79916
46795
125445
605035


In [77]:
ratings = pd.concat([ratings[ratings['rating'] == 1], ratings[ratings['rating'] == 2], ratings[ratings['rating'] == 3], ratings[ratings['rating'] == 4][:125000]])

In [78]:
user_id_arr = np.array(ratings['user_id'].value_counts().tolist()) 

In [79]:
len(np.where(user_id_arr >= 10)[0])

922

In [80]:
for threshold in tqdm([0, 10]):
    user_id_arr_ = np.array(ratings['user_id'].value_counts().index.tolist())[np.where(user_id_arr >= threshold)[0]]
    user_id_df = pd.DataFrame(columns=['user_id'])
    user_id_df['user_id'] = user_id_arr_.tolist()
    print(len(user_id_df))
    ratings_eliminate_user = pd.merge(ratings, user_id_df, how='inner', on='user_id')
    item_ids_in_detail_set = set(details['id'].unique())
    item_ids_in_overall_set = set(overalls['id'].unique())
    item_ids_in_id_set = set(ids['id'].unique())
    item_ids_in_rating_set = set(ratings_eliminate_user['item_id'].unique())
    intersec_item_ids = item_ids_in_detail_set.intersection(item_ids_in_overall_set).intersection(item_ids_in_id_set).intersection(item_ids_in_rating_set)
    item_index_df = pd.DataFrame(columns=['item_id', 'item index'])
    item_index_df['item_id'] = list(intersec_item_ids)
    item_index_df['item index'] = np.arange(len(intersec_item_ids)).tolist()
    print(len(item_index_df))
    details_convert_item_id = pd.merge(details, item_index_df, how='inner', left_on='id', right_on='item_id')
    details_convert_item_id = details_convert_item_id.drop(columns=['item_id'])

    overalls_convert_item_id = pd.merge(overalls, item_index_df, how='inner', left_on='id', right_on='item_id')
    overalls_convert_item_id = overalls_convert_item_id.drop(columns=['item_id'])

    ratings_convert_item_id = pd.merge(ratings_eliminate_user, item_index_df, how='inner', left_on='item_id', right_on='item_id')

    ids_convert_item_id = pd.merge(ids, item_index_df, how='inner', left_on='id', right_on='item_id')
    ids_convert_item_id = ids_convert_item_id.drop(columns=['item_id'])
    
    user_index_df = pd.DataFrame(columns=['user_id', 'user index'])
    user_index_df['user_id'] = ratings_convert_item_id['user_id'].value_counts().index.tolist()
    user_index_df['user index'] = np.arange(len(ratings_convert_item_id['user_id'].value_counts().tolist())).tolist()

    ratings_convert_user_id = pd.merge(ratings_convert_item_id, user_index_df, how='inner', on='user_id')

    if not os.path.exists('./data/datasets/item_{}'.format(threshold)):
        os.makedirs('./data/datasets/item_{}'.format(threshold))
    
    if not os.path.exists('./data/datasets/rating_{}'.format(threshold)):
        os.makedirs('./data/datasets/rating_{}'.format(threshold))
    
    ratings_convert_user_id.to_csv(item_rating_list_out_path.format(threshold), sep=',', encoding='utf-8', index=False)
    details_convert_item_id.to_csv(item_detail_list_out_path.format(threshold), sep=',', encoding='utf-8', index=False)
    ids_convert_item_id.to_csv(item_id_list_out_path.format(threshold), sep=',', encoding='utf-8', index=False)
    overalls_convert_item_id.to_csv(item_overall_list_out_path.format(threshold), sep=',', encoding='utf-8', index=False)


  0%|          | 0/2 [00:00<?, ?it/s]

266782
16928


 50%|█████     | 1/2 [00:50<00:50, 50.61s/it]

922
4940


100%|██████████| 2/2 [01:01<00:00, 30.59s/it]
